第三章主要介绍以下内容：
- 将不同种类的真实世界的数据表示成`pytorch`的`tensors`
- 处理各种数据类型，包括电子表格，时间序列，文本，图像和医学影像
- 从文件中加载数据
- 将`data`转换成`tensor`
- 根据不同需求调整张量的形状

# Tabular data

机器学习任务中最常见到的可能就是在表格、CSV文件或者数据库中的数据，一般是表格形式呈现。
- 首先，表格里面样本之间是独立的，样本在表格中的顺序没有什么意义。
- 表格中的列数据类型大部分是不一样的，有些列是整形，有些是字符串，而pytorch的tensor是同质的，一般是浮点型，所以我们要做的是将这些数据都转换成tensor支持的类型
网上可以很容易的获取大量的表格数据，[点此获取](https://github.com/caesar0301/awesome-public-datasets.)

本次我们选用白酒品质数据集进行说明，数据集在`./data/winequality-white.csv`

`python`提供给我们几种快速加载CSV文件的方法，如下所示：
- `python`的CSV模块
- `Numpy`方法
- `Pandas`方法*(最省时间最省内存)*

鉴于我们之前提过`Numpy`和`tensor`之间可以相互转换，所以我们使用`Numpy`进行过渡生成`tensor`

In [4]:
import numpy as np
wine_path = "./data/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy, wineq_numpy.shape

(array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
        [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
        [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
        ...,
        [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
        [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
        [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32),
 (4898, 12))

以上，dtype规定元素数据类型，delimiter规定分隔符，skiprows表示跳过第一行，第一行是列名集合

In [3]:
# 打印出列名集合
import csv
col_list = next(csv.reader(open(wine_path), delimiter=';'))
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

得到`numpy`的`ndarray`，然后再转换成`tensor`：

In [6]:
import torch
wineq = torch.from_numpy(wineq_numpy)
wineq, wineq.shape, wineq.type()

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  0.4500,  8.8000,  6.0000],
         [ 6.3000,  0.3000,  0.3400,  ...,  0.4900,  9.5000,  6.0000],
         [ 8.1000,  0.2800,  0.4000,  ...,  0.4400, 10.1000,  6.0000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  0.4600,  9.4000,  6.0000],
         [ 5.5000,  0.2900,  0.3000,  ...,  0.3800, 12.8000,  7.0000],
         [ 6.0000,  0.2100,  0.3800,  ...,  0.3200, 11.8000,  6.0000]]),
 torch.Size([4898, 12]),
 'torch.FloatTensor')

**注意**
你需要特别注意以下三种数值：
1. **`continuous values`**: 这种数值是最常见的，往往就是连续的数值，数值之间可以比较大小，也可以进行数学运算，比如身高体重的值
2. **`ordinal values`**: 这种数值是表示顺序的值，数值之间可以比较大小，但是不可以进行数学运算，例如容量大小有三个值small, medium和large,对应于1， 2和 3。可以进行比较大小，但是数学运算没有意义。
3. **`categorical values`**: 没有比较大小或者数学运算操作，单纯表示的是分类值，目的是将样本划分进几个类别

通常情况下，数据最后一列`quality`作为`ground truth`，我们需要将它单独抽出来作为一个新的`tensor`备用:

In [7]:
data = wineq[:, :-1]        #抽取所有行，抽取除去最后一列的所有列
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [8]:
target = wineq[:, -1]       # 抽取所有行，抽取最后一列
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

如果遇到需要把`target`当成标签来看的情况，我们可以有以下方法操作：
1. 把`target`每一个类别当成一个整数
2. 对每一个`target`进行`one-hot`编码

关于`one-hot`编码，可以这么理解，已知目前有5类，标号1， 2， 3， 4， 5，我们利用长度为 5 的向量依次表示这5类，重点在于这些向量的元素值不是0就是1

类别|one-hot编码
:-:|:-:
1|1 0 0 0 0
2|0 1 0 0 0
3|0 0 1 0 0
4|0 0 0 1 0
5|0 0 0 0 1

我们也可以打乱映射顺序，不影响最后结果。这样就可以将类别进行编码，便于计算。

我们可以使用scatter_方法实现one-hot编码：

In [10]:
# 我们可以使用scatter_方法实现one-hot编码
target_onehot = torch.zeros(target.shape[0], 10)      # 有多少类就有多少行，选择多长的向量来表示就可以指定
target = target.long()     #这一句是必须的，要把target变成long()
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

scatter_方法的参数解释：
- The dimension along which the following two arguments are specified 
- A column tensor indicating the indices of the elements to scatter
- A tensor containing the elements to scatter or a single scalar to scatter (1, in this case) 

观察`unsqueeze()`方法的作用：

In [14]:
a = torch.tensor([1, 2, 3, 4, 5, 6])
a_unsqueezed = a.unsqueeze(1)
a, a_unsqueezed

(tensor([1, 2, 3, 4, 5, 6]), tensor([[1],
         [2],
         [3],
         [4],
         [5],
         [6]]))

回到表格数据data：

求每一列的均值：

In [15]:
# 表示均值
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

求每一列的方差

In [16]:
# 方差
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

归一化数据：

In [17]:
# normalize
data_normalized = (data - data_mean)/torch.sqrt(data_var)
data_normalized

tensor([[ 1.7209e-01, -8.1764e-02,  2.1325e-01,  ..., -1.2468e+00,
         -3.4914e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7991e-02,  ...,  7.3992e-01,
          1.3467e-03, -8.2418e-01],
        [ 1.4756e+00,  1.7448e-02,  5.4378e-01,  ...,  4.7502e-01,
         -4.3677e-01, -3.3662e-01],
        ...,
        [-4.2042e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3131e+00,
         -2.6152e-01, -9.0544e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0048e+00,
         -9.6250e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7502e-01,
         -1.4882e+00,  1.0448e+00]])

现在我们开始处理数据，我们的思路是，先根据`target`将所有样本分成几类，对数据的分布以及规律有个基本的掌握：

我们的分类标准是，`target`得分小于等于3的归为`bad_data`；得分（3， 7）之间的归为`mid_data`；得分超过或等于7的归为`good_data`

这里我们要把target和数字作比较，然后截取符合要求的数据，提供几个torch的方法来实现这个功能：

实现目标|方法
:-:|:-:
a大于等于b|torch.ge(a, b)
a小于等于b|torch.le(a, b)
a大于b|torch.gt(a, b)
a小于b|torch.lt(a, b)

In [18]:
bad_data = data[torch.le(target, 3)] 
mid_data = data[torch.gt(target, 3) & torch.lt(target, 7)] 
good_data = data[torch.ge(target, 7)]

bad_mean = torch.mean(bad_data, dim=0) 
mid_mean = torch.mean(mid_data, dim=0) 
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):    
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


初步观察得，`bad_data`里面`total sulfur dioxide`含量高于其他的样本，那我们可以给定阈值来区分坏酒和好酒

In [19]:
total_sulfur_threshold = 141.83 
total_sulfur_data = data[:,6] 
# 预测是好酒
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [20]:
actual_indexes = torch.gt(target, 5)
# 事实上好酒
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [21]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item() 
n_predicted = torch.sum(predicted_indexes).item() 
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

可以看出来我们这么简单的添加阈值判断好酒坏酒，有2018瓶酒被预测正确

虽然这种方法非常简单且结果还算不错，我们心知肚明这肯定不是判断的最好的方法，但这只是开始，等以后接触神经网络和其他算法，我们就可以实现更加准确的预测

# Time series

上面我们了解了表格数据的形式和处理方法，知道了表格数据的每一行都是独立的，没有任何编码信息包含了行的顺序关系。

我们这一节主要讲述时间序列的数据，借用上面表格数据很好理解，也就是我们可以新增一个`Year`列，这样我们就可以看到酒的品质`quality`随着时间`year`变化的关系了，但是上面的数据集里面没有。

为了更好地了解时间序列，我们重新找了一个数据集：[华盛顿自行车共享系统数据集](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset.)

In [1]:
import numpy as np
import torch
bike_numpy = np.loadtxt("./data/hour-fixed.csv", dtype=np.float32, delimiter=',', skiprows=1, converters={1: lambda x: float(x[-2:])})
# converter的含义是对选中的列进行处理，比如这里对第二列进行截取最后两位，然后强制类型转换
bikes = torch.from_numpy(bike_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [2]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

从上面我们可以看到数据总共有17520条，每一条17列。

观察`CSV`文件可知，`row`是按照每一天来排列，每一天又按照24个小时来排列，所以我们将`bikes`reshape一下， 让其含有三个维度，分别是`（day, hour, 17 columns）`

In [22]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])    #返回一个新的tensor
daily_bikes

tensor([[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00,
          1.3000e+01, 1.6000e+01],
         [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00,
          3.2000e+01, 4.0000e+01],
         [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00,
          2.7000e+01, 3.2000e+01],
         ...,
         [2.2000e+01, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00,
          3.1000e+01, 3.4000e+01],
         [2.3000e+01, 1.0000e+00, 1.0000e+00,  ..., 1.1000e+01,
          1.7000e+01, 2.8000e+01],
         [2.4000e+01, 1.0000e+00, 1.0000e+00,  ..., 1.5000e+01,
          2.4000e+01, 3.9000e+01]],

        [[2.5000e+01, 2.0000e+00, 1.0000e+00,  ..., 4.0000e+00,
          1.3000e+01, 1.7000e+01],
         [2.6000e+01, 2.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.6000e+01, 1.7000e+01],
         [2.7000e+01, 2.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          8.0000e+00, 9.0000e+00],
         ...,
         [4.5000e+01, 2.0000e+00, 1.0000e+00,  ..., 1.1000e+01,
          2.000

In [23]:
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

以上数据进行一个讲解，我们得到的`daily_bikes`主要结构是第一维表示哪一天， 第二维表示一天之内的24个小时，第三维表示17个特征。

我们通过`stride()`可以知道从当前天数到下一天需要跳过408个数据，这好理解，因为一天内24h，一小时17特征，`17*24=408`。

从当前小时到下一个小时，需要跳过17个数据，因为一小时17个特征值。从当前特征跳到下一特征只需要移动1。但是我们希望从当前小时跳到下一小时是连续的，只需要移动1。所以我们需要对第二维和第三维进行转置：

In [24]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

接下来我们考虑一些特定的特征，比如说`weather-situation`，可以看到这是一个分类特征，一共四个类：1表示最好的天气，4表示最差的天气。我们前面说过，如果将其看成是分类标签的话，可以对它进行`one-hot`编码，为了使操作简单，我们只考虑一天之内的数据：

In [7]:
first_day = bikes[:24].long()    #全部转换成长整型
first_day

tensor([[  1,   1,   1,   0,   1,   0,   0,   6,   0,   1,   0,   0,   0,   0,
           3,  13,  16],
        [  2,   1,   1,   0,   1,   1,   0,   6,   0,   1,   0,   0,   0,   0,
           8,  32,  40],
        [  3,   1,   1,   0,   1,   2,   0,   6,   0,   1,   0,   0,   0,   0,
           5,  27,  32],
        [  4,   1,   1,   0,   1,   3,   0,   6,   0,   1,   0,   0,   0,   0,
           3,  10,  13],
        [  5,   1,   1,   0,   1,   4,   0,   6,   0,   1,   0,   0,   0,   0,
           0,   1,   1],
        [  6,   1,   1,   0,   1,   5,   0,   6,   0,   2,   0,   0,   0,   0,
           0,   1,   1],
        [  7,   1,   1,   0,   1,   6,   0,   6,   0,   1,   0,   0,   0,   0,
           2,   0,   2],
        [  8,   1,   1,   0,   1,   7,   0,   6,   0,   1,   0,   0,   0,   0,
           1,   2,   3],
        [  9,   1,   1,   0,   1,   8,   0,   6,   0,   1,   0,   0,   0,   0,
           1,   7,   8],
        [ 10,   1,   1,   0,   1,   9,   0,   6,   0,   1,   0, 

In [11]:
weather_onehot = torch.zeros(first_day.shape[0], 4)      #指定one-hot的维度
first_day[:, 9]    #我们想要得到hours和weather的情况，所以只需要取出weather列即可

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [13]:
# 进行one-hot编码
weather_onehot.scatter_(dim=1, index=first_day[:, 9].unsqueeze(1)-1, value=1.0)
#这里相当于对于一个全0的tensor，按照index来逐个进行赋值

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

将生成的编码与原始数据合并起来，使用`torch.cat()`:

In [14]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]  #合并完取出第一行打印出来看看

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

对于转置后的`tensor`如何进行`one-hot`编码呢，转置后的`tensor`是`daily_bikes`，`shape`是`(730*17*24)`。我们还是首先创建一个全0的tensor，形状是`(704, 4， 24)`，因为到时候生成的tensor和原tensor合并，所以只改变中间的，其余维度不变。

In [15]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [19]:
#进行one-hot编码
daily_weather_onehot.scatter_(dim=1, index=daily_bikes[:,9,:].long().unsqueeze(1)-1, value=1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [25]:
#合并两个tensor
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
daily_bikes.shape

torch.Size([730, 21, 24])

以上作`one-hot`编码，是因为我们将`weather`列当成标签，但是我们之前说过也可以直接当成数值来看待，因为数值之间有大小关系，对应于天气好坏的比较，所以我们接下来介绍当成数值如何操作。

我们说过，纯数值列我们可以进行归一化，便于模型训练，常见的归一化有以下两种：
```
1. min_max归一化
2. 0-1归一化
```

我们拿数据集里面的temperature列作说明：

In [ ]:
# min_max归一化
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = (daily_bikes[:, 10, :] - temp_min)/(temp.max - temp.min)

或者使用下面的方法，等效：

In [ ]:
# 0-1归一化
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = (daily_bikes[:, 10, :] - torch.mean(temp)) / torch.std(temp)

以上就是对于time series数据的简单介绍，我们看完之后如果知道时间数据是如何操作的就很不错了，常见的时间序列数据是文本和语音。

# Text

# Images

# Volumetric data